<a href="https://colab.research.google.com/github/MarceloClaro/QuantumDeepClassifier/blob/main/QuantumDeepClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Explicação dos Comandos de Instalação**

Os comandos listados utilizam o gerenciador de pacotes **pip** para instalar bibliotecas específicas que são usadas em computação quântica, aprendizado de máquina e visualização de dados. Abaixo, explico cada uma delas:

---

### **1. `!pip install qiskit`**
- **O que é o Qiskit?**
  - O Qiskit é uma biblioteca de código aberto para computação quântica desenvolvida pela IBM. Ele permite:
    - Criar, simular e executar circuitos quânticos.
    - Realizar experimentos em computadores quânticos reais da IBM Quantum ou simuladores locais.
    - Trabalhar com algoritmos quânticos, como **VQE**, **QAOA** e **Shor**.

- **Principais Módulos do Qiskit:**
  - **`qiskit.circuit`**: Criação de circuitos quânticos.
  - **`qiskit.aer`**: Simulação de circuitos quânticos.
  - **`qiskit.ibmq`**: Conexão com dispositivos quânticos reais na nuvem.
  - **`qiskit.visualization`**: Visualização de circuitos quânticos.

- **Para que serve?**
  - Desenvolver aplicações quânticas em áreas como criptografia, otimização, química e aprendizado de máquina.

---

### **2. `!pip install pennylane`**
- **O que é o PennyLane?**
  - O PennyLane é uma biblioteca de código aberto para **computação quântica diferencial**. Ele integra computação quântica com aprendizado de máquina (AM) e frameworks como PyTorch, TensorFlow e NumPy.

- **Principais Recursos do PennyLane:**
  - Suporte a **diferenciação automática**: Permite calcular gradientes de circuitos quânticos para ajustar parâmetros durante o treinamento.
  - Compatibilidade com dispositivos quânticos reais e simuladores.
  - Ferramentas para implementar algoritmos híbridos (quânticos e clássicos).

- **Para que serve?**
  - Criar modelos híbridos que combinam redes neurais e circuitos quânticos.
  - Aplicar aprendizado de máquina quântico em tarefas como classificação, regressão e redução de dimensionalidade.

---

### **3. `!pip install tensorflow-quantum`**
- **O que é o TensorFlow Quantum (TFQ)?**
  - O TensorFlow Quantum é uma extensão do TensorFlow que facilita a integração de circuitos quânticos com aprendizado de máquina clássico.
  - Ele é desenvolvido pela Google AI e permite:
    - Construir e treinar modelos híbridos (quânticos e clássicos).
    - Simular circuitos quânticos dentro do fluxo de trabalho do TensorFlow.

- **Principais Recursos do TFQ:**
  - **Integração com TensorFlow**: Usado com outras APIs TensorFlow, como `tf.keras` e `tf.data`.
  - **Diferenciação automática** para parâmetros de circuitos quânticos.
  - **Simuladores de dispositivos quânticos** otimizados para desempenho.

- **Para que serve?**
  - Desenvolver modelos híbridos para tarefas de aprendizado supervisionado, não supervisionado e reforçado.
  - Aplicar computação quântica em AM em escala usando a infraestrutura do TensorFlow.

---

### **4. `!pip install matplotlib`**
- **O que é o Matplotlib?**
  - O Matplotlib é uma biblioteca de Python usada para criar gráficos estáticos, interativos e animados.

- **Principais Recursos do Matplotlib:**
  - Criação de gráficos de linha, dispersão, histogramas, barras e muito mais.
  - Personalização total de estilos, rótulos, títulos e cores.
  - Compatibilidade com notebooks interativos (Jupyter Notebook, Google Colab).

- **Para que serve?**
  - Visualizar resultados de simulações e treinamentos de modelos quânticos e clássicos.
  - Interpretar dados e métricas por meio de gráficos.

---

### **5. `!pip install pillow`**
- **O que é o Pillow?**
  - O Pillow (ou PIL, Python Imaging Library) é uma biblioteca de manipulação de imagens.

- **Principais Recursos do Pillow:**
  - Abrir, modificar e salvar imagens em vários formatos (JPEG, PNG, BMP, etc.).
  - Redimensionar, cortar, converter para escala de cinza e normalizar imagens.
  - Integrar pipelines de visão computacional.

- **Para que serve?**
  - Pré-processar dados de imagens antes de usá-los em modelos quânticos ou clássicos.
  - Trabalhar com datasets de imagens em tarefas de classificação ou detecção de objetos.

---

### **Resumo e Propósito Geral**
Esses pacotes combinados permitem a construção de modelos de aprendizado de máquina quânticos e híbridos. Com eles, você pode:
1. **Simular e executar circuitos quânticos**: Usando Qiskit e PennyLane.
2. **Integrar computação quântica e aprendizado de máquina clássico**: Com PennyLane e TensorFlow Quantum.
3. **Pré-processar e visualizar dados**: Usando Matplotlib e Pillow.



In [ ]:
!pip install qiskit
!pip install pennylane
!pip install tensorflow-quantum
!pip install matplotlib
!pip install pillow


### Código Completo para Carregar, Visualizar e Salvar as Imagens no Drive

Aqui está o código ajustado para solicitar o arquivo `melanomas.zip`, organizar, visualizar as imagens das classes, e salvar os dados processados na pasta "quantum" no Google Drive.

---

#### **Código Ajustado**

```python
import zipfile
import os
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import drive

# 1. Montar o Google Drive
drive.mount('/content/drive')
output_path = "/content/drive/My Drive/quantum"
os.makedirs(output_path, exist_ok=True)

# 2. Solicitar o arquivo melanomas.zip
zip_path = input("Insira o caminho do arquivo melanomas.zip no seu Google Drive: ")
if not os.path.exists(zip_path):
    print("Arquivo não encontrado! Verifique o caminho e tente novamente.")
else:
    # 3. Extrair o arquivo ZIP
    extract_path = "/content/melanomas"
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # 4. Listar os arquivos extraídos
    extracted_files = []
    for root, dirs, files in os.walk(extract_path):
        for file in files:
            if file.endswith((".jpg", ".png")):  # Apenas imagens
                extracted_files.append(os.path.join(root, file))

    print(f"Número total de imagens: {len(extracted_files)}")
    print("Exemplo de arquivos extraídos:", extracted_files[:5])

    # 5. Função para visualizar imagens por classe
    def visualize_images(files, n=5):
        """Visualizar as primeiras N imagens de cada classe"""
        classes = {}
        for file in files:
            class_name = os.path.basename(os.path.dirname(file))
            if class_name not in classes:
                classes[class_name] = []
            classes[class_name].append(file)

        for class_name, images in classes.items():
            print(f"Classe: {class_name} | Total de imagens: {len(images)}")
            plt.figure(figsize=(15, 5))
            plt.suptitle(f"Exemplos da classe: {class_name}")
            for i, img_path in enumerate(images[:n]):
                img = Image.open(img_path)
                plt.subplot(1, n, i + 1)
                plt.imshow(img)
                plt.axis("off")
                plt.title(f"{class_name} {i+1}")
            plt.show()

    # 6. Visualizar as imagens
    visualize_images(extracted_files, n=5)

    # 7. Salvar as imagens redimensionadas no Google Drive
    image_size = (64, 64)  # Tamanho padrão para redimensionar
    classes = {}
    for file in extracted_files:
        class_name = os.path.basename(os.path.dirname(file))
        class_path = os.path.join(output_path, class_name)
        os.makedirs(class_path, exist_ok=True)

        img = Image.open(file)
        img_resized = img.resize(image_size)  # Redimensionar imagem
        save_path = os.path.join(class_path, os.path.basename(file))
        img_resized.save(save_path)

        if class_name not in classes:
            classes[class_name] = []
        classes[class_name].append(save_path)

    print(f"Imagens redimensionadas e salvas em {output_path}")

    # 8. Mostrar exemplos das imagens redimensionadas
    visualize_images(extracted_files, n=5)
```

---

#### **Passo a Passo do Código**
1. **Montar o Google Drive**:
   - Monta o Google Drive no Colab para armazenar os dados processados.
2. **Solicitar o Arquivo `melanomas.zip`**:
   - O código pede ao usuário o caminho do arquivo ZIP no Drive.
   - Exemplo: `/content/drive/My Drive/melanomas.zip`.
3. **Extrair Imagens**:
   - As imagens são extraídas para a pasta temporária `/content/melanomas`.
   - Apenas arquivos com extensões `.jpg` e `.png` são incluídos.
4. **Visualizar Imagens por Classe**:
   - As imagens são agrupadas com base no nome das subpastas (representando as classes).
   - As primeiras `n` imagens de cada classe são exibidas em gráficos.
5. **Salvar Imagens no Google Drive**:
   - As imagens são redimensionadas para 64x64 pixels.
   - Elas são organizadas em pastas no Drive, em `My Drive/quantum/<nome_da_classe>`.
6. **Exibir Imagens Redimensionadas**:
   - Após salvar, as imagens redimensionadas são exibidas novamente.

---

#### **Exemplo de Execução**
- O código irá:
  - Solicitar o arquivo `melanomas.zip`.
  - Extrair e organizar as imagens por classe.
  - Exibir as primeiras 5 imagens de cada classe.
  - Salvar as imagens redimensionadas no Google Drive para posterior uso.



In [ ]:
import zipfile
import os
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import drive

# 1. Montar o Google Drive
drive.mount('/content/drive')
output_path = "/content/drive/My Drive/quantum"
os.makedirs(output_path, exist_ok=True)

# 2. Solicitar o arquivo melanomas.zip
zip_path = input("Insira o caminho do arquivo melanomas.zip no seu Google Drive: ")
if not os.path.exists(zip_path):
    print("Arquivo não encontrado! Verifique o caminho e tente novamente.")
else:
    # 3. Extrair o arquivo ZIP
    extract_path = "/content/melanomas"
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

    # 4. Listar os arquivos extraídos
    extracted_files = []
    for root, dirs, files in os.walk(extract_path):
        for file in files:
            if file.endswith((".jpg", ".png")):  # Apenas imagens
                extracted_files.append(os.path.join(root, file))

    print(f"Número total de imagens: {len(extracted_files)}")
    print("Exemplo de arquivos extraídos:", extracted_files[:5])

    # 5. Função para visualizar imagens por classe
    def visualize_images(files, n=5):
        """Visualizar as primeiras N imagens de cada classe"""
        classes = {}
        for file in files:
            class_name = os.path.basename(os.path.dirname(file))
            if class_name not in classes:
                classes[class_name] = []
            classes[class_name].append(file)

        for class_name, images in classes.items():
            print(f"Classe: {class_name} | Total de imagens: {len(images)}")
            plt.figure(figsize=(15, 5))
            plt.suptitle(f"Exemplos da classe: {class_name}")
            for i, img_path in enumerate(images[:n]):
                img = Image.open(img_path)
                plt.subplot(1, n, i + 1)
                plt.imshow(img)
                plt.axis("off")
                plt.title(f"{class_name} {i+1}")
            plt.show()

    # 6. Visualizar as imagens
    visualize_images(extracted_files, n=5)

    # 7. Salvar as imagens redimensionadas no Google Drive
    image_size = (64, 64)  # Tamanho padrão para redimensionar
    classes = {}
    for file in extracted_files:
        class_name = os.path.basename(os.path.dirname(file))
        class_path = os.path.join(output_path, class_name)
        os.makedirs(class_path, exist_ok=True)

        img = Image.open(file)
        img_resized = img.resize(image_size)  # Redimensionar imagem
        save_path = os.path.join(class_path, os.path.basename(file))
        img_resized.save(save_path)

        if class_name not in classes:
            classes[class_name] = []
        classes[class_name].append(save_path)

    print(f"Imagens redimensionadas e salvas em {output_path}")

    # 8. Mostrar exemplos das imagens redimensionadas
    visualize_images(extracted_files, n=5)


Sim, podemos ajustar o código para:

1. **Visualizar a imagem original e redimensionada**.
2. **Salvar os dados redimensionados em um DataFrame e exportá-lo para um arquivo, se necessário.**

---

### Código Ajustado para Visualização e Criação de DataFrame

```python
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Função para processar e redimensionar imagens
def process_and_visualize_image(image_path, resize_to=(64, 64)):
    """Processa a imagem, redimensiona e visualiza"""
    # Abrir a imagem
    image = Image.open(image_path)
    
    # Redimensionar a imagem
    image_resized = image.resize(resize_to)
    
    # Converter para array normalizado
    image_array = np.array(image_resized) / 255.0

    # Visualizar a imagem original e redimensionada
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Imagem Original")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(image_resized)
    plt.title(f"Imagem Redimensionada {resize_to}")
    plt.axis("off")
    plt.show()

    return image_array

# Exemplo com a primeira imagem
sample_image_path = extracted_files[0]
processed_image = process_and_visualize_image(sample_image_path)

# Criar um DataFrame com os dados redimensionados
def create_dataframe(image_paths, resize_to=(64, 64)):
    """Processa imagens e cria um DataFrame com arrays normalizados"""
    data = []
    labels = []
    for image_path in image_paths:
        # Processar a imagem
        image_resized = Image.open(image_path).resize(resize_to)
        image_array = np.array(image_resized).flatten() / 255.0  # Achatar a matriz
        
        # Obter o rótulo da classe
        label = os.path.basename(os.path.dirname(image_path))
        
        # Adicionar ao dataset
        data.append(image_array)
        labels.append(label)
    
    # Criar o DataFrame
    df = pd.DataFrame(data)
    df['label'] = labels
    return df

# Criar o DataFrame com todas as imagens
df = create_dataframe(extracted_files)

# Visualizar parte do DataFrame
print(df.head())

# Salvar o DataFrame em um arquivo CSV
output_path = "processed_images.csv"
df.to_csv(output_path, index=False)
print(f"DataFrame salvo em: {output_path}")
```

---

### Explicação do Código

1. **Visualização da Imagem**:
   - A função `process_and_visualize_image` exibe a imagem original e a redimensionada lado a lado.

2. **Criação do DataFrame**:
   - Cada imagem é convertida para um array 1D (achatar a matriz).
   - Os arrays são armazenados como linhas no DataFrame, com uma coluna adicional para os rótulos das classes.

3. **Exportação para CSV**:
   - O DataFrame é salvo em um arquivo CSV (`processed_images.csv`), para ser reutilizado posteriormente.

---

### Resultados Esperados
- Você verá a imagem original e redimensionada para confirmar o processo de redimensionamento.
- O DataFrame conterá os dados de todas as imagens redimensionadas e seus rótulos de classe.
- O arquivo CSV permitirá reutilizar os dados sem necessidade de processar as imagens novamente.

Teste o código e informe se precisar de mais ajustes! 😊

In [ ]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')
output_dir = "/content/drive/My Drive/quantum"
os.makedirs(output_dir, exist_ok=True)

# Função para processar e visualizar uma única imagem
def process_and_visualize_image(image_path, resize_to=(64, 64)):
    """Processa a imagem, redimensiona e visualiza"""
    # Abrir a imagem
    image = Image.open(image_path)

    # Redimensionar a imagem
    image_resized = image.resize(resize_to)

    # Converter para array normalizado
    image_array = np.array(image_resized) / 255.0

    # Visualizar a imagem original e redimensionada
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.title("Imagem Original")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(image_resized)
    plt.title(f"Imagem Redimensionada {resize_to}")
    plt.axis("off")
    plt.show()

    return image_array

# Exemplo com a primeira imagem
if len(extracted_files) > 0:
    sample_image_path = extracted_files[0]
    processed_image = process_and_visualize_image(sample_image_path)
else:
    print("Nenhuma imagem foi encontrada para processamento.")

# Função para processar todas as imagens e criar um DataFrame
def create_dataframe_and_save_images(image_paths, resize_to=(64, 64), save_dir=output_dir):
    """Processa imagens, cria um DataFrame e salva imagens redimensionadas"""
    data = []
    labels = []
    for image_path in image_paths:
        # Processar a imagem
        image = Image.open(image_path).resize(resize_to)
        image_array = np.array(image).flatten() / 255.0  # Achatar a matriz

        # Obter o rótulo da classe
        label = os.path.basename(os.path.dirname(image_path))

        # Salvar imagem redimensionada na pasta "quantum"
        class_dir = os.path.join(save_dir, label)
        os.makedirs(class_dir, exist_ok=True)
        image_save_path = os.path.join(class_dir, os.path.basename(image_path))
        image.save(image_save_path)

        # Adicionar ao dataset
        data.append(image_array)
        labels.append(label)

    # Criar o DataFrame
    df = pd.DataFrame(data)
    df['label'] = labels

    # Salvar o DataFrame na pasta "quantum"
    df_output_path = os.path.join(save_dir, "processed_images.csv")
    df.to_csv(df_output_path, index=False)
    print(f"DataFrame salvo em: {df_output_path}")

    return df

# Processar imagens e salvar no Google Drive
if len(extracted_files) > 0:
    df = create_dataframe_and_save_images(extracted_files)
    print("Primeiras linhas do DataFrame:")
    print(df.head())
else:
    print("Nenhuma imagem encontrada para criar o DataFrame.")


### Código Ajustado para Visualizar, Normalizar Imagens e Salvar no Drive

Este é o código modificado para salvar o DataFrame e imagens no Google Drive dentro da pasta "quantum". Ele processa as imagens, exibe as visualizações necessárias, cria um DataFrame e salva os resultados.

---

```python
import glob
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')
output_dir = "/content/drive/My Drive/quantum"
os.makedirs(output_dir, exist_ok=True)

# Caminho das pastas para cada classe
class_dirs = [os.path.join(extract_path, "benigno"), os.path.join(extract_path, "maligno")]
image_size = (64, 64)

# Função para processar, visualizar e salvar imagens
def process_images_with_visualization_and_save(image_paths, image_size, n_visualizations=5, save_dir=output_dir):
    images = []
    labels = []
    visualized = 0  # Contador de imagens visualizadas
    
    for image_path in image_paths:
        # Identificar a classe a partir do caminho
        label = os.path.basename(os.path.dirname(image_path))
        
        # Abrir, redimensionar e normalizar a imagem
        image = Image.open(image_path)
        image_resized = image.resize(image_size)
        image_array = np.array(image_resized) / 255.0  # Normalização
        
        # Salvar imagem redimensionada na pasta "quantum"
        class_dir = os.path.join(save_dir, label)
        os.makedirs(class_dir, exist_ok=True)
        image_save_path = os.path.join(class_dir, os.path.basename(image_path))
        image_resized.save(image_save_path)
        
        # Adicionar ao dataset
        images.append(image_array)
        labels.append(label)
        
        # Visualizar algumas imagens
        if visualized < n_visualizations:
            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(image)
            plt.title("Original")
            plt.axis("off")
            plt.subplot(1, 2, 2)
            plt.imshow(image_resized)
            plt.title(f"Redimensionada ({image_size}) e Normalizada")
            plt.axis("off")
            plt.show()
            print(f"Array Normalizado ({image_array.shape}):")
            print(image_array[:5, :5, 0])  # Mostrar parte da matriz normalizada
            visualized += 1

    return np.array(images), labels

# Coletar todas as imagens do diretório
all_image_paths = [img for class_dir in class_dirs for img in glob.glob(f"{class_dir}/*.jpg")]

# Processar imagens com visualização e salvar
processed_images, labels = process_images_with_visualization_and_save(all_image_paths, image_size)

# Função para criar e salvar o DataFrame
def create_dataframe_and_save(images, labels, save_path):
    flattened_images = [img.flatten() for img in images]  # Achatar as imagens
    df = pd.DataFrame(flattened_images)
    df['label'] = labels
    
    # Salvar o DataFrame no caminho especificado
    df.to_csv(save_path, index=False)
    print(f"DataFrame salvo em: {save_path}")
    return df

# Criar e salvar o DataFrame
df_output_path = os.path.join(output_dir, "processed_images_with_labels.csv")
df = create_dataframe_and_save(processed_images, labels, df_output_path)

# Visualizar o DataFrame
print("Amostra do DataFrame:")
print(df.head())
```

---

### **Explicação do Código**

1. **Montar o Google Drive**:
   - Monta o Google Drive no Colab para salvar os arquivos na pasta "quantum".

2. **Visualização e Processamento das Imagens**:
   - Processa as imagens para redimensioná-las a um tamanho padrão de \(64 \times 64\).
   - Normaliza os valores dos pixels para o intervalo \([0, 1]\).
   - Exibe as imagens originais e redimensionadas lado a lado.

3. **Salvar Imagens Processadas**:
   - As imagens redimensionadas são salvas na pasta "quantum", organizadas por subpastas das classes.

4. **Criação do DataFrame**:
   - Cria um DataFrame com:
     - Vetores achatados das imagens (uma linha por imagem).
     - Rótulos das classes como uma coluna separada.
   - Salva o DataFrame no formato CSV na pasta "quantum".

---

### **Resultados Esperados**

1. **Visualização de Imagens**:
   - As imagens originais e redimensionadas são exibidas no Colab.

2. **Pasta `quantum` no Drive**:
   - Contém subpastas para cada classe, com as imagens redimensionadas salvas.
   - Um arquivo `processed_images_with_labels.csv` com os arrays das imagens e os rótulos.

3. **Exemplo de DataFrame**:
   - Um DataFrame com as imagens processadas e normalizadas, incluindo os rótulos.



In [ ]:
import glob
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

# Montar o Google Drive
drive.mount('/content/drive')
output_dir = "/content/drive/My Drive/quantum"
os.makedirs(output_dir, exist_ok=True)

# Caminho das pastas para cada classe
class_dirs = [os.path.join(extract_path, "benigno"), os.path.join(extract_path, "maligno")]
image_size = (64, 64)

# Função para processar, visualizar e salvar imagens
def process_images_with_visualization_and_save(image_paths, image_size, n_visualizations=5, save_dir=output_dir):
    images = []
    labels = []
    visualized = 0  # Contador de imagens visualizadas

    for image_path in image_paths:
        # Identificar a classe a partir do caminho
        label = os.path.basename(os.path.dirname(image_path))

        # Abrir, redimensionar e normalizar a imagem
        image = Image.open(image_path)
        image_resized = image.resize(image_size)
        image_array = np.array(image_resized) / 255.0  # Normalização

        # Salvar imagem redimensionada na pasta "quantum"
        class_dir = os.path.join(save_dir, label)
        os.makedirs(class_dir, exist_ok=True)
        image_save_path = os.path.join(class_dir, os.path.basename(image_path))
        image_resized.save(image_save_path)

        # Adicionar ao dataset
        images.append(image_array)
        labels.append(label)

        # Visualizar algumas imagens
        if visualized < n_visualizations:
            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(image)
            plt.title("Original")
            plt.axis("off")
            plt.subplot(1, 2, 2)
            plt.imshow(image_resized)
            plt.title(f"Redimensionada ({image_size}) e Normalizada")
            plt.axis("off")
            plt.show()
            print(f"Array Normalizado ({image_array.shape}):")
            print(image_array[:5, :5, 0])  # Mostrar parte da matriz normalizada
            visualized += 1

    return np.array(images), labels

# Coletar todas as imagens do diretório
all_image_paths = [img for class_dir in class_dirs for img in glob.glob(f"{class_dir}/*.jpg")]

# Processar imagens com visualização e salvar
processed_images, labels = process_images_with_visualization_and_save(all_image_paths, image_size)

# Função para criar e salvar o DataFrame
def create_dataframe_and_save(images, labels, save_path):
    flattened_images = [img.flatten() for img in images]  # Achatar as imagens
    df = pd.DataFrame(flattened_images)
    df['label'] = labels

    # Salvar o DataFrame no caminho especificado
    df.to_csv(save_path, index=False)
    print(f"DataFrame salvo em: {save_path}")
    return df

# Criar e salvar o DataFrame
df_output_path = os.path.join(output_dir, "processed_images_with_labels.csv")
df = create_dataframe_and_save(processed_images, labels, df_output_path)

# Visualizar o DataFrame
print("Amostra do DataFrame:")
print(df.head())


Aqui está o código ajustado para salvar na pasta `quantum` no Google Drive e incluir todas as funcionalidades mencionadas: visualização, normalização, divisão dos dados e salvamento em DataFrames.

---

### Código Ajustado

```python
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

# Montar o Google Drive e criar a pasta 'quantum'
drive.mount('/content/drive')
output_dir = "/content/drive/My Drive/quantum"
os.makedirs(output_dir, exist_ok=True)

# Convertendo rótulos de texto para valores numéricos
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Transformando imagens em vetores unidimensionais
flattened_images = processed_images.reshape(processed_images.shape[0], -1)

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(flattened_images, encoded_labels, test_size=0.2, random_state=42)

# Visualização de amostras do conjunto de treino
def visualize_data_split(X, y, label_encoder, n=5):
    """Visualiza algumas imagens do conjunto de treino ou teste"""
    for i in range(n):
        image = X[i].reshape(64, 64, 3)  # Restaurar formato original
        label = label_encoder.inverse_transform([y[i]])[0]  # Decodificar rótulo numérico para texto
        plt.imshow(image)
        plt.title(f"Classe: {label}")
        plt.axis("off")
        plt.show()
        print(f"Matriz Normalizada (amostra {i+1}):")
        print(image[:5, :5, 0])  # Exibe uma parte da matriz normalizada

# Visualizar amostras do conjunto de treino
print("Amostras do conjunto de treino:")
visualize_data_split(X_train, y_train, label_encoder, n=5)

# Criar DataFrame com os dados de treino e teste
def create_dataframe(X, y, label_encoder):
    """Cria um DataFrame com as imagens achatadas e os rótulos"""
    df = pd.DataFrame(X)
    df['label'] = label_encoder.inverse_transform(y)  # Adicionar os rótulos decodificados
    return df

# Criar DataFrames para treino e teste
train_df = create_dataframe(X_train, y_train, label_encoder)
test_df = create_dataframe(X_test, y_test, label_encoder)

# Salvar os DataFrames em arquivos CSV na pasta quantum
train_csv_path = os.path.join(output_dir, "train_data.csv")
test_csv_path = os.path.join(output_dir, "test_data.csv")

train_df.to_csv(train_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

print(f"DataFrame de treino salvo em: {train_csv_path}")
print(f"DataFrame de teste salvo em: {test_csv_path}")

# Exibir amostras dos DataFrames
print("Amostra do DataFrame de treino:")
print(train_df.head())
print("Amostra do DataFrame de teste:")
print(test_df.head())
```

---

### **O Que Este Código Faz**

1. **Montagem do Google Drive**:
   - Monta o Google Drive para salvar os arquivos na pasta `quantum`.

2. **Processamento de Rótulos e Dados**:
   - Converte rótulos textuais em valores numéricos usando `LabelEncoder`.
   - Redimensiona e achata as imagens processadas para vetores 1D.

3. **Divisão de Dados**:
   - Divide os dados em conjuntos de treino e teste usando `train_test_split`.

4. **Visualização**:
   - Exibe algumas imagens do conjunto de treino com seus rótulos decodificados.
   - Mostra parte das matrizes normalizadas de cada imagem.

5. **Criação de DataFrames**:
   - Cria DataFrames para os conjuntos de treino e teste.
   - Adiciona os rótulos decodificados como uma coluna.

6. **Salvamento**:
   - Salva os DataFrames como arquivos CSV (`train_data.csv` e `test_data.csv`) na pasta `quantum` no Google Drive.

---

### **Resultados Esperados**

1. **Visualizações**:
   - Exibição de 5 imagens do conjunto de treino com seus rótulos decodificados.
   - Matrizes normalizadas das imagens para análise.

2. **Arquivos CSV**:
   - Arquivos `train_data.csv` e `test_data.csv` contendo os dados processados e os rótulos, salvos na pasta `quantum`.

3. **Exemplo de DataFrame**:
   - DataFrames contendo:
     - Colunas com os valores achatados dos pixels.
     - Coluna `label` com os rótulos das imagens.

---



In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive

# Montar o Google Drive e criar a pasta 'quantum'
drive.mount('/content/drive')
output_dir = "/content/drive/My Drive/quantum"
os.makedirs(output_dir, exist_ok=True)

# Convertendo rótulos de texto para valores numéricos
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Transformando imagens em vetores unidimensionais
flattened_images = processed_images.reshape(processed_images.shape[0], -1)

# Dividindo os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(flattened_images, encoded_labels, test_size=0.2, random_state=42)

# Visualização de amostras do conjunto de treino
def visualize_data_split(X, y, label_encoder, n=5):
    """Visualiza algumas imagens do conjunto de treino ou teste"""
    for i in range(n):
        image = X[i].reshape(64, 64, 3)  # Restaurar formato original
        label = label_encoder.inverse_transform([y[i]])[0]  # Decodificar rótulo numérico para texto
        plt.imshow(image)
        plt.title(f"Classe: {label}")
        plt.axis("off")
        plt.show()
        print(f"Matriz Normalizada (amostra {i+1}):")
        print(image[:5, :5, 0])  # Exibe uma parte da matriz normalizada

# Visualizar amostras do conjunto de treino
print("Amostras do conjunto de treino:")
visualize_data_split(X_train, y_train, label_encoder, n=5)

# Criar DataFrame com os dados de treino e teste
def create_dataframe(X, y, label_encoder):
    """Cria um DataFrame com as imagens achatadas e os rótulos"""
    df = pd.DataFrame(X)
    df['label'] = label_encoder.inverse_transform(y)  # Adicionar os rótulos decodificados
    return df

# Criar DataFrames para treino e teste
train_df = create_dataframe(X_train, y_train, label_encoder)
test_df = create_dataframe(X_test, y_test, label_encoder)

# Salvar os DataFrames em arquivos CSV na pasta quantum
train_csv_path = os.path.join(output_dir, "train_data.csv")
test_csv_path = os.path.join(output_dir, "test_data.csv")

train_df.to_csv(train_csv_path, index=False)
test_df.to_csv(test_csv_path, index=False)

print(f"DataFrame de treino salvo em: {train_csv_path}")
print(f"DataFrame de teste salvo em: {test_csv_path}")

# Exibir amostras dos DataFrames
print("Amostra do DataFrame de treino:")
print(train_df.head())
print("Amostra do DataFrame de teste:")
print(test_df.head())


Aqui está o código ajustado para salvar os resultados no Google Drive na pasta **quantum**, com todas as explicações e etapas necessárias para análise e visualização avançada:

---

### Código Ajustado

```python
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from google.colab import drive
import pennylane as qml

# Montar o Google Drive e criar a pasta 'quantum'
drive.mount('/content/drive')
output_dir = "/content/drive/My Drive/quantum"
os.makedirs(output_dir, exist_ok=True)

# Configuração do dispositivo quântico
n_qubits = 10
dev = qml.device("default.qubit", wires=n_qubits)

# Função de embedding quântico
def data_embedding(features, wires):
    for i, wire in enumerate(wires):
        qml.RY(features[i], wires=wire)

# Função do modelo quântico
def quantum_model(weights, features):
    data_embedding(features, range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

# QNode com Pennylane
n_layers = 4
weights_shape = (n_layers, n_qubits)
weights = np.random.random(weights_shape)

@qml.qnode(dev)
def circuit(weights, features):
    return quantum_model(weights, features)

# Processamento das amostras
def process_samples(X, y, n_qubits, weights):
    data = []
    for i in range(len(X)):
        features = X[i][:n_qubits]
        features = np.pad(features, (0, n_qubits - len(features))) if len(features) < n_qubits else features[:n_qubits]
        prediction = float(circuit(weights, features))
        residual = prediction - y[i]
        data.append({"features": features.tolist(), "label": y[i], "prediction": prediction, "residual": residual})
    return pd.DataFrame(data)

# Visualizar e salvar histogramas
def plot_histograms(df, output_dir):
    plt.figure(figsize=(12, 6))
    for label in df['label'].unique():
        subset = df[df['label'] == label]
        plt.hist(subset['prediction'], bins=20, alpha=0.7, label=f"Classe {label}")
    plt.title("Distribuição das Previsões por Classe")
    plt.xlabel("Previsão")
    plt.ylabel("Frequência")
    plt.legend()
    plt.savefig(os.path.join(output_dir, "histogram_predictions.png"))
    plt.show()

# Visualizar circuitos
def plot_circuits(X, y, n_samples=2):
    for label in np.unique(y):
        indices = np.where(y == label)[0][:n_samples]
        for idx in indices:
            features = X[idx][:n_qubits]
            features = np.pad(features, (0, n_qubits - len(features))) if len(features) < n_qubits else features[:n_qubits]
            print(f"Circuito para a amostra {idx} (Classe {label}):")
            drawer = qml.draw(circuit)(weights, features)
            print(drawer)
            print()

# Dividir os dados e criar DataFrames
def create_train_test_data(processed_images, labels, n_qubits):
    # Codificar rótulos
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)

    # Flatten imagens e dividir em treino/teste
    flattened_images = processed_images.reshape(processed_images.shape[0], -1)
    X_train, X_test, y_train, y_test = train_test_split(flattened_images, encoded_labels, test_size=0.2, random_state=42)

    # Criar DataFrames
    train_df = process_samples(X_train, y_train, n_qubits, weights)
    test_df = process_samples(X_test, y_test, n_qubits, weights)

    # Salvar DataFrames no Google Drive
    train_df.to_csv(os.path.join(output_dir, "train_data_quantum.csv"), index=False)
    test_df.to_csv(os.path.join(output_dir, "test_data_quantum.csv"), index=False)
    print(f"DataFrames salvos na pasta 'quantum':")
    print("- train_data_quantum.csv")
    print("- test_data_quantum.csv")

    return train_df, test_df

# Simulação de processamento (substitua processed_images e labels pelos dados reais)
# Exemplo hipotético para rodar o código:
processed_images = np.random.random((100, 64, 64, 3))  # Substitua com imagens reais
labels = np.random.choice(["benigno", "maligno"], size=100)  # Substitua com rótulos reais

# Criar os DataFrames
train_df, test_df = create_train_test_data(processed_images, labels, n_qubits)

# Exibir amostras e histogramas
print("Amostra do DataFrame de treino:")
print(train_df.head())
print("\nAmostra do DataFrame de teste:")
print(test_df.head())
plot_histograms(train_df, output_dir)

# Visualizar circuitos para algumas amostras
plot_circuits(processed_images, labels)
```

---

### **O Que Esse Código Faz**

1. **Montagem do Google Drive**:
   - Salva os arquivos na pasta `quantum` no Google Drive.

2. **Processamento Quântico**:
   - Cada amostra de imagem é normalizada, redimensionada e processada por um circuito quântico.
   - O circuito aplica rotações \( RY \) para incorporar os dados e usa camadas de entanglement para capturar correlações.

3. **Divisão de Dados**:
   - As imagens e os rótulos são divididos em conjuntos de treino e teste.

4. **Criação de DataFrames**:
   - Cria DataFrames para treino e teste contendo:
     - `features`: Dados normalizados e achatados.
     - `label`: Classe da amostra.
     - `prediction`: Saída do circuito quântico.
     - `residual`: Diferença entre previsão e rótulo.

5. **Salvamento de Resultados**:
   - Salva os DataFrames (`train_data_quantum.csv`, `test_data_quantum.csv`) na pasta `quantum`.

6. **Visualização**:
   - Exibe histogramas das previsões para cada classe.
   - Mostra o circuito quântico usado para processar algumas amostras.

---

### **Resultados Esperados**

1. **Arquivos Salvos**:
   - `train_data_quantum.csv` e `test_data_quantum.csv` contendo os resultados processados.

2. **Visualizações**:
   - Histogramas mostrando a distribuição das previsões por classe.
   - Circuitos desenhados para algumas amostras, mostrando as rotações \( RY \) e as camadas de entanglement.

---



In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from google.colab import drive
import pennylane as qml

# Montar o Google Drive e criar a pasta 'quantum'
drive.mount('/content/drive')
output_dir = "/content/drive/My Drive/quantum"
os.makedirs(output_dir, exist_ok=True)

# Configuração do dispositivo quântico
n_qubits = 10
dev = qml.device("default.qubit", wires=n_qubits)

# Função de embedding quântico
def data_embedding(features, wires):
    for i, wire in enumerate(wires):
        qml.RY(features[i], wires=wire)

# Função do modelo quântico
def quantum_model(weights, features):
    data_embedding(features, range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

# QNode com Pennylane
n_layers = 4
weights_shape = (n_layers, n_qubits)
weights = np.random.random(weights_shape)

@qml.qnode(dev)
def circuit(weights, features):
    return quantum_model(weights, features)

# Processamento das amostras
def process_samples(X, y, n_qubits, weights):
    data = []
    for i in range(len(X)):
        features = X[i][:n_qubits]
        features = np.pad(features, (0, n_qubits - len(features))) if len(features) < n_qubits else features[:n_qubits]
        prediction = float(circuit(weights, features))
        residual = prediction - y[i]
        data.append({"features": features.tolist(), "label": y[i], "prediction": prediction, "residual": residual})
    return pd.DataFrame(data)

# Visualizar e salvar histogramas
def plot_histograms(df, output_dir):
    plt.figure(figsize=(12, 6))
    for label in df['label'].unique():
        subset = df[df['label'] == label]
        plt.hist(subset['prediction'], bins=20, alpha=0.7, label=f"Classe {label}")
    plt.title("Distribuição das Previsões por Classe")
    plt.xlabel("Previsão")
    plt.ylabel("Frequência")
    plt.legend()
    plt.savefig(os.path.join(output_dir, "histogram_predictions.png"))
    plt.show()

# Visualizar circuitos
def plot_circuits(X, y, n_samples=2):
    for label in np.unique(y):
        indices = np.where(y == label)[0][:n_samples]
        for idx in indices:
            features = X[idx][:n_qubits]
            features = np.pad(features, (0, n_qubits - len(features))) if len(features) < n_qubits else features[:n_qubits]
            print(f"Circuito para a amostra {idx} (Classe {label}):")
            drawer = qml.draw(circuit)(weights, features)
            print(drawer)
            print()

# Dividir os dados e criar DataFrames
def create_train_test_data(processed_images, labels, n_qubits):
    # Codificar rótulos
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)

    # Flatten imagens e dividir em treino/teste
    flattened_images = processed_images.reshape(processed_images.shape[0], -1)
    X_train, X_test, y_train, y_test = train_test_split(flattened_images, encoded_labels, test_size=0.2, random_state=42)

    # Criar DataFrames
    train_df = process_samples(X_train, y_train, n_qubits, weights)
    test_df = process_samples(X_test, y_test, n_qubits, weights)

    # Salvar DataFrames no Google Drive
    train_df.to_csv(os.path.join(output_dir, "train_data_quantum.csv"), index=False)
    test_df.to_csv(os.path.join(output_dir, "test_data_quantum.csv"), index=False)
    print(f"DataFrames salvos na pasta 'quantum':")
    print("- train_data_quantum.csv")
    print("- test_data_quantum.csv")

    return train_df, test_df

# Simulação de processamento (substitua processed_images e labels pelos dados reais)
# Exemplo hipotético para rodar o código:
processed_images = np.random.random((100, 64, 64, 3))  # Substitua com imagens reais
labels = np.random.choice(["benigno", "maligno"], size=100)  # Substitua com rótulos reais

# Criar os DataFrames
train_df, test_df = create_train_test_data(processed_images, labels, n_qubits)

# Exibir amostras e histogramas
print("Amostra do DataFrame de treino:")
print(train_df.head())
print("\nAmostra do DataFrame de teste:")
print(test_df.head())
plot_histograms(train_df, output_dir)

# Visualizar circuitos para algumas amostras
plot_circuits(processed_images, labels)


Aqui está o código ajustado que inclui **Early Stopping** e **parametrização dos embeddings** para um melhor equilíbrio durante o treinamento e avaliação do modelo quântico, salvando todos os resultados na pasta `quantum` no Google Drive.

---

### **Código com Early Stopping e Parametrização**

```python
import os
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pennylane.optimize import AdamOptimizer
from google.colab import drive

# Montar o Google Drive e criar a pasta 'quantum'
drive.mount('/content/drive')
output_dir = "/content/drive/My Drive/quantum"
os.makedirs(output_dir, exist_ok=True)

# Configuração do dispositivo quântico
n_qubits = 10
n_layers = 4  # Camadas do modelo quântico
dev = qml.device("default.qubit", wires=n_qubits)

# Função de embedding quântico parametrizada
def data_embedding(features, wires, scale=1.0):
    for i, wire in enumerate(wires):
        qml.RY(features[i] * scale, wires=wire)

# Função do modelo quântico
def quantum_model(weights, features, scale=1.0):
    data_embedding(features, range(n_qubits), scale)
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

# QNode
weights_shape = (n_layers, n_qubits)

@qml.qnode(dev)
def circuit(weights, features, scale=1.0):
    return quantum_model(weights, features, scale)

# Função de custo
def cost(weights, X, y, scale=1.0):
    loss = 0
    for i in range(len(X)):
        pred = circuit(weights, X[i], scale)
        loss += (pred - y[i])**2
    return loss / len(X)

# Inicialização de pesos
weights = np.random.uniform(low=-0.1, high=0.1, size=weights_shape, requires_grad=True)

# Configuração do otimizador
opt = AdamOptimizer(stepsize=0.01)
steps = 100  # Número de iterações
early_stopping_patience = 10  # Critério de Early Stopping
min_delta = 1e-4  # Tolerância mínima para melhoria no custo

# Dados ajustados
X_train_resized = X_train[:, :n_qubits]
X_test_resized = X_test[:, :n_qubits]

# Variáveis para Early Stopping
best_weights = None
best_test_cost = float('inf')
no_improvement_count = 0

# Treinamento
train_costs = []
test_costs = []

for step in range(steps):
    weights = opt.step(lambda w: cost(w, X_train_resized, y_train, scale=1.0), weights)
    train_cost = cost(weights, X_train_resized, y_train, scale=1.0)
    test_cost = cost(weights, X_test_resized, y_test, scale=1.0)
    
    train_costs.append(train_cost)
    test_costs.append(test_cost)
    
    # Early Stopping
    if test_cost < best_test_cost - min_delta:
        best_test_cost = test_cost
        best_weights = weights
        no_improvement_count = 0
    else:
        no_improvement_count += 1
    
    if no_improvement_count >= early_stopping_patience:
        print(f"Parada antecipada no passo {step}. Melhor custo no teste: {best_test_cost:.4f}")
        break
    
    if step % 10 == 0:
        print(f"Step {step}/{steps}: Train Cost = {train_cost:.4f} | Test Cost = {test_cost:.4f}")

# Usar os melhores pesos encontrados
weights = best_weights

# Avaliação final
final_train_cost = cost(weights, X_train_resized, y_train, scale=1.0)
final_test_cost = cost(weights, X_test_resized, y_test, scale=1.0)
print(f"Custo final no conjunto de treino: {final_train_cost:.4f}")
print(f"Custo final no conjunto de teste: {final_test_cost:.4f}")

# Previsões
y_train_pred = [float(circuit(weights, x)) for x in X_train_resized]
y_test_pred = [float(circuit(weights, x)) for x in X_test_resized]

# Criação do DataFrame com resíduos
df_results = pd.DataFrame({
    "train_features": [x.tolist() for x in X_train_resized],
    "train_labels": y_train,
    "train_predictions": y_train_pred,
    "train_residuals": [pred - y for pred, y in zip(y_train_pred, y_train)],
    "test_features": [x.tolist() for x in X_test_resized],
    "test_labels": y_test,
    "test_predictions": y_test_pred,
    "test_residuals": [pred - y for pred, y in zip(y_test_pred, y_test)]
})

# Salvar os resultados no Google Drive
results_path = os.path.join(output_dir, "quantum_model_results.csv")
df_results.to_csv(results_path, index=False)
print(f"Resultados salvos no arquivo: {results_path}")

# Visualizações

# 1. Gráfico de custo durante o treinamento
plt.figure(figsize=(10, 6))
plt.plot(range(len(train_costs)), train_costs, label="Custo de Treinamento", color="blue")
plt.plot(range(len(test_costs)), test_costs, label="Custo de Teste", color="orange")
plt.title("Evolução do Custo Durante o Treinamento")
plt.xlabel("Passos")
plt.ylabel("Custo")
plt.legend()
plt.savefig(os.path.join(output_dir, "training_costs.png"))
plt.show()

# 2. Gráfico de dispersão (predição vs rótulo)
plt.figure(figsize=(10, 6))
plt.scatter(y_train, y_train_pred, alpha=0.7, label="Treino", color="blue")
plt.scatter(y_test, y_test_pred, alpha=0.7, label="Teste", color="orange")
plt.title("Dispersão: Previsão vs Rótulo")
plt.xlabel("Rótulo Verdadeiro")
plt.ylabel("Previsão")
plt.legend()
plt.savefig(os.path.join(output_dir, "scatter_predictions.png"))
plt.show()

# 3. Histograma de resíduos
plt.figure(figsize=(10, 6))
plt.hist(df_results["train_residuals"], bins=20, alpha=0.7, label="Treino", color="blue")
plt.hist(df_results["test_residuals"], bins=20, alpha=0.7, label="Teste", color="orange")
plt.title("Distribuição dos Resíduos")
plt.xlabel("Resíduo")
plt.ylabel("Frequência")
plt.legend()
plt.savefig(os.path.join(output_dir, "residuals_histogram.png"))
plt.show()
```

---

### **Explicações Adicionais**

1. **Early Stopping**:
   - Interrompe o treinamento se não houver melhoria significativa no custo do conjunto de teste por um número definido de iterações (`early_stopping_patience`).

2. **Parametrização dos Embeddings**:
   - Permite ajustar o impacto dos valores de entrada nos qubits, usando o parâmetro `scale`.

3. **Resultados Salvos**:
   - Arquivo `quantum_model_results.csv` com previsões, resíduos e features.
   - Gráficos salvos na pasta `quantum`:
     - `training_costs.png`
     - `scatter_predictions.png`
     - `residuals_histogram.png`



O código em execução combina **aprendizado quântico** e **otimização clássica** para treinar um modelo quântico, salvando os resultados e visualizações no Google Drive, dentro da pasta `quantum`. Aqui está o que ocorre em cada etapa e o que esperar dos resultados:

---

### **1. Configuração do Ambiente**
- Monta o Google Drive e cria a pasta `quantum` para armazenar os resultados.
- Configura o dispositivo quântico `default.qubit` com 10 qubits e 4 camadas no circuito.

---

### **2. Embedding Quântico**
- Converte os dados clássicos (`features`) em rotações \( RY \), mapeando-os para o espaço quântico.
- O parâmetro `scale` ajusta a amplitude das rotações, permitindo balancear a influência dos dados.

---

### **3. Modelo e Circuito**
- **Camadas Entangled**:
  - Introduzem correlações quânticas entre os qubits, aumentando a capacidade do modelo.
- **Expectativa \( \langle Z \rangle \)**:
  - Mede a projeção dos estados quânticos no eixo \( Z \), gerando uma saída contínua para cada entrada.

---

### **4. Função de Custo**
- Calcula o **Erro Quadrático Médio (MSE)** entre as previsões e os rótulos verdadeiros.
- É usado pelo otimizador para ajustar os pesos do modelo.

---

### **5. Otimização com Early Stopping**
- **AdamOptimizer**:
  - Atualiza os pesos do circuito para minimizar o custo.
- **Early Stopping**:
  - Monitora o custo no conjunto de teste.
  - Para o treinamento se não houver melhoria após 10 iterações consecutivas (definido por `early_stopping_patience`).

---

### **6. Avaliação**
- Após o treinamento, calcula o custo final para os conjuntos de treino e teste.
- Usa os pesos ajustados para gerar previsões para ambos os conjuntos.

---

### **7. Resultados no DataFrame**
- O `df_results` inclui:
  - **`train_features`** e **`test_features`**: Dados de entrada (após redimensionamento).
  - **`train_labels`** e **`test_labels`**: Rótulos verdadeiros.
  - **`train_predictions`** e **`test_predictions`**: Previsões do modelo.
  - **`train_residuals`** e **`test_residuals`**: Diferença entre previsão e rótulo verdadeiro (erros).
- Salva o DataFrame como `quantum_model_results.csv` na pasta `quantum`.

---

### **8. Visualizações**
#### **Gráfico de Custo**
- Mostra a evolução do custo durante o treinamento.
- **Azul**: Custo no conjunto de treino.
- **Laranja**: Custo no conjunto de teste.
  
#### **Gráfico de Dispersão**
- Compara previsões e rótulos verdadeiros.
- Padrão esperado:
  - Pontos próximos à linha \( y = x \) indicam previsões precisas.

#### **Histograma de Resíduos**
- Mostra a distribuição dos erros (previsão - rótulo).
- Resíduos próximos de zero indicam um bom ajuste.

---

### **Resultados Esperados**
1. **Treinamento**:
   - Redução do custo nos conjuntos de treino e teste ao longo das iterações.
   - Early Stopping pode ocorrer se o custo no teste estabilizar.

2. **Visualizações**:
   - Gráficos salvos na pasta `quantum`:
     - `training_costs.png`: Evolução do custo.
     - `scatter_predictions.png`: Dispersão previsão vs rótulo.
     - `residuals_histogram.png`: Distribuição dos resíduos.

3. **Arquivo CSV**:
   - **`quantum_model_results.csv`**: Contém previsões, resíduos e features, permitindo análise detalhada.

Se o processamento ainda está em andamento, aguarde os gráficos e o arquivo `quantum_model_results.csv` no Drive para validação dos resultados. 😊

In [ ]:
import os
import pennylane as qml
from pennylane import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pennylane.optimize import AdamOptimizer
from google.colab import drive

# Montar o Google Drive e criar a pasta 'quantum'
drive.mount('/content/drive')
output_dir = "/content/drive/My Drive/quantum"
os.makedirs(output_dir, exist_ok=True)

# Configuração do dispositivo quântico
n_qubits = 10
n_layers = 4  # Camadas do modelo quântico
dev = qml.device("default.qubit", wires=n_qubits)

# Função de embedding quântico parametrizada
def data_embedding(features, wires, scale=1.0):
    for i, wire in enumerate(wires):
        qml.RY(features[i] * scale, wires=wire)

# Função do modelo quântico
def quantum_model(weights, features, scale=1.0):
    data_embedding(features, range(n_qubits), scale)
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))

# QNode
weights_shape = (n_layers, n_qubits)

@qml.qnode(dev)
def circuit(weights, features, scale=1.0):
    return quantum_model(weights, features, scale)

# Função de custo
def cost(weights, X, y, scale=1.0):
    loss = 0
    for i in range(len(X)):
        pred = circuit(weights, X[i], scale)
        loss += (pred - y[i])**2
    return loss / len(X)

# Inicialização de pesos
weights = np.random.uniform(low=-0.1, high=0.1, size=weights_shape, requires_grad=True)

# Configuração do otimizador
opt = AdamOptimizer(stepsize=0.01)
steps = 100  # Número de iterações
early_stopping_patience = 10  # Critério de Early Stopping
min_delta = 1e-4  # Tolerância mínima para melhoria no custo

# Dados ajustados
X_train_resized = X_train[:, :n_qubits]
X_test_resized = X_test[:, :n_qubits]

# Variáveis para Early Stopping
best_weights = None
best_test_cost = float('inf')
no_improvement_count = 0

# Treinamento
train_costs = []
test_costs = []

for step in range(steps):
    weights = opt.step(lambda w: cost(w, X_train_resized, y_train, scale=1.0), weights)
    train_cost = cost(weights, X_train_resized, y_train, scale=1.0)
    test_cost = cost(weights, X_test_resized, y_test, scale=1.0)

    train_costs.append(train_cost)
    test_costs.append(test_cost)

    # Early Stopping
    if test_cost < best_test_cost - min_delta:
        best_test_cost = test_cost
        best_weights = weights
        no_improvement_count = 0
    else:
        no_improvement_count += 1

    if no_improvement_count >= early_stopping_patience:
        print(f"Parada antecipada no passo {step}. Melhor custo no teste: {best_test_cost:.4f}")
        break

    if step % 10 == 0:
        print(f"Step {step}/{steps}: Train Cost = {train_cost:.4f} | Test Cost = {test_cost:.4f}")

# Usar os melhores pesos encontrados
weights = best_weights

# Avaliação final
final_train_cost = cost(weights, X_train_resized, y_train, scale=1.0)
final_test_cost = cost(weights, X_test_resized, y_test, scale=1.0)
print(f"Custo final no conjunto de treino: {final_train_cost:.4f}")
print(f"Custo final no conjunto de teste: {final_test_cost:.4f}")

# Previsões
y_train_pred = [float(circuit(weights, x)) for x in X_train_resized]
y_test_pred = [float(circuit(weights, x)) for x in X_test_resized]

# Criação do DataFrame com resíduos
df_results = pd.DataFrame({
    "train_features": [x.tolist() for x in X_train_resized],
    "train_labels": y_train,
    "train_predictions": y_train_pred,
    "train_residuals": [pred - y for pred, y in zip(y_train_pred, y_train)],
    "test_features": [x.tolist() for x in X_test_resized],
    "test_labels": y_test,
    "test_predictions": y_test_pred,
    "test_residuals": [pred - y for pred, y in zip(y_test_pred, y_test)]
})

# Salvar os resultados no Google Drive
results_path = os.path.join(output_dir, "quantum_model_results.csv")
df_results.to_csv(results_path, index=False)
print(f"Resultados salvos no arquivo: {results_path}")

# Visualizações

# 1. Gráfico de custo durante o treinamento
plt.figure(figsize=(10, 6))
plt.plot(range(len(train_costs)), train_costs, label="Custo de Treinamento", color="blue")
plt.plot(range(len(test_costs)), test_costs, label="Custo de Teste", color="orange")
plt.title("Evolução do Custo Durante o Treinamento")
plt.xlabel("Passos")
plt.ylabel("Custo")
plt.legend()
plt.savefig(os.path.join(output_dir, "training_costs.png"))
plt.show()

# 2. Gráfico de dispersão (predição vs rótulo)
plt.figure(figsize=(10, 6))
plt.scatter(y_train, y_train_pred, alpha=0.7, label="Treino", color="blue")
plt.scatter(y_test, y_test_pred, alpha=0.7, label="Teste", color="orange")
plt.title("Dispersão: Previsão vs Rótulo")
plt.xlabel("Rótulo Verdadeiro")
plt.ylabel("Previsão")
plt.legend()
plt.savefig(os.path.join(output_dir, "scatter_predictions.png"))
plt.show()

# 3. Histograma de resíduos
plt.figure(figsize=(10, 6))
plt.hist(df_results["train_residuals"], bins=20, alpha=0.7, label="Treino", color="blue")
plt.hist(df_results["test_residuals"], bins=20, alpha=0.7, label="Teste", color="orange")
plt.title("Distribuição dos Resíduos")
plt.xlabel("Resíduo")
plt.ylabel("Frequência")
plt.legend()
plt.savefig(os.path.join(output_dir, "residuals_histogram.png"))
plt.show()
